<a href="https://colab.research.google.com/github/munni-annapareddy123/Climate-Trend-Analysis/blob/main/Climate_Trend_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # Choose GlobalTemperature.zip


Saving GlobalTemperature.zip to GlobalTemperature (1).zip


In [ ]:
import zipfile

with zipfile.ZipFile('GlobalTemperature.zip', 'r') as zip_ref:
    zip_ref.extractall('./')  # Extracts everything in Colab's current folder

!ls  # Check extracted contents


 GlobalTemperature	      GlobalTemperature.zip
'GlobalTemperature (1).zip'   sample_data


In [ ]:
!ls GlobalTemperature/


GlobalLandTemperaturesByCity.csv       GlobalLandTemperaturesByState.csv
GlobalLandTemperaturesByCountry.csv    GlobalTemperatures.csv
GlobalLandTemperaturesByMajorCity.csv


In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv('GlobalTemperature/GlobalTemperatures.csv')

# Display first 5 rows
df.head()


,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Basic info about columns and types
df.info()

# Summary statistics
df.describe()

# Check missing values
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 9 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   dt                                         3192 non-null   object 
 1   LandAverageTemperature                     3180 non-null   float64
 2   LandAverageTemperatureUncertainty          3180 non-null   float64
 3   LandMaxTemperature                         1992 non-null   float64
 4   LandMaxTemperatureUncertainty              1992 non-null   float64
 5   LandMinTemperature                         1992 non-null   float64
 6   LandMinTemperatureUncertainty              1992 non-null   float64
 7   LandAndOceanAverageTemperature             1992 non-null   float64
 8   LandAndOceanAverageTemperatureUncertainty  1992 non-null   float64
dtypes: float64(8), object(1)
memory usage: 224.6+ KB


,0
dt,0
LandAverageTemperature,12
LandAverageTemperatureUncertainty,12
LandMaxTemperature,1200
LandMaxTemperatureUncertainty,1200
LandMinTemperature,1200
LandMinTemperatureUncertainty,1200
LandAndOceanAverageTemperature,1200
LandAndOceanAverageTemperatureUncertainty,1200


In [ ]:
!git --version


git version 2.34.1


In [ ]:
!git clone https://github.com/munni-annapareddy123/Climate-Trend-Analysis.git


Cloning into 'Climate-Trend-Analysis'...


In [ ]:
%cd Climate-Trend-Analysis


/content/Climate-Trend-Analysis


In [ ]:
!git config --global user.email "annapareddylikitha2006@gmail.com"


In [ ]:
!git config --global user.name "Likhitha Reddy"

In [ ]:
!cp "/content/Climate Trend Analysis.ipynb" .


cp: cannot stat '/content/Climate Trend Analysis.ipynb': No such file or directory


In [ ]:
!ls /content


Climate-Trend-Analysis	sample_data


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving GlobalTemperature.zip to GlobalTemperature.zip


In [ ]:
!ls /content


Climate-Trend-Analysis	sample_data
